# Titanic competition with TensorFlow Decision Forests

This notebook will take you through the steps needed to train a baseline Gradient Boosted Trees Model using TensorFlow Decision Forests and creating a submission on the Titanic competition. 

This notebook shows:

1. How to do some basic pre-processing. For example, the passenger names will be tokenized, and ticket names will be splitted in parts.
1. How to train a Gradient Boosted Trees (GBT) with default parameters
1. How to train a GBT with improved default parameters
1. How to tune the parameters of a GBTs
1. How to train and ensemble many GBTs

Gradient Boosted Trees（梯度提升树，简称GBT或GBDT）是一种集成学习方法，主要用于回归和分类任务。它通过**集成多个弱学习器（通常是决策树）**来形成一个强学习器。这里是一个信息密度高的解释：

核心思想
迭代训练： 每一轮训练一个新树，用来纠正之前模型的残差（即预测误差）。

加权求和： 所有树的预测结果加权相加，得到最终预测值。

梯度下降： 利用损失函数的梯度，指导每棵新树如何拟合残差。

# Imports dependencies

In [ ]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf

print(f"Found TF-DF {tfdf.__version__}")

# Load dataset

In [ ]:
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
serving_df = pd.read_csv("/kaggle/input/titanic/test.csv")

train_df.head(10)

# Prepare dataset

We will apply the following transformations on the dataset.

1. Tokenize the names. For example, "Braund, Mr. Owen Harris" will become ["Braund", "Mr.", "Owen", "Harris"].
2. Extract any prefix in the ticket. For example ticket "STON/O2. 3101282" will become "STON/O2." and 3101282.

In [ ]:
def preprocess(df):
    df = df.copy()
    
    def normalize_name(x):
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    
    def ticket_number(x):
        return x.split(" ")[-1]
        
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])
    
    df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)                     
    return df
    
preprocessed_train_df = preprocess(train_df)
preprocessed_serving_df = preprocess(serving_df)

preprocessed_train_df.head(5)

Let's keep the list of the input features of the model. Notably, we don't want to train our model on the "PassengerId" and "Ticket" features.

In [ ]:
input_features = list(preprocessed_train_df.columns)
input_features.remove("Ticket")
input_features.remove("PassengerId")
input_features.remove("Survived")
#input_features.remove("Ticket_number")

print(f"Input features: {input_features}")

# Convert Pandas dataset to TensorFlow Dataset

In [ ]:
def tokenize_names(features, labels=None):
    """Divite the names into tokens. TF-DF can consume text tokens natively."""
    features["Name"] =  tf.strings.split(features["Name"])
    return features, labels

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df,label="Survived").map(tokenize_names)
serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(tokenize_names)

features 是一个字典，键是列名，值是张量。

tf.strings.split(features["Name"]) 会将姓名字符串按空格拆分成列表（词 tokens），如 "Smith, John" -> ["Smith,", "John"]

TF-DF 可以直接使用 token 列作为文本特征，不像一般的模型需要做 embedding。

最后返回修改后的 features 和标签 labels（如果有）。

pd_dataframe_to_tf_dataset(...)：把 pandas 的 DataFrame 转成 TensorFlow 的数据集。

label="Survived"：指定目标变量。

.map(tokenize_names)：对数据集的每一条记录应用 tokenize_names 函数，即进行姓名 token 化。

# Train model with default parameters

### Train model

First, we are training a GradientBoostedTreesModel model with the default parameters.

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # Very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    random_seed=1234,
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

# Train model with improved default parameters

Now you'll use some specific parameters when creating the GBT model

In [ ]:
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # Very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    
    #num_trees=2000,
    
    # Only for GBT.
    # A bit slower, but great to understand the model.
    # compute_permutation_variable_importance=True,
    
    # Change the default hyper-parameters
    # hyperparameter_template="benchmark_rank1@v1",
    
    #num_trees=1000,
    #tuner=tuner
    
    min_examples=1,
    categorical_algorithm="RANDOM",
    #max_depth=4,
    shrinkage=0.05,
    #num_candidate_attributes_ratio=0.2,
    split_axis="SPARSE_OBLIQUE",
    sparse_oblique_normalization="MIN_MAX",
    sparse_oblique_num_projections_exponent=2.0,
    num_trees=2000,
    #validation_ratio=0.0,
    random_seed=1234,
    
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

### 超参数（重点部分）
min_examples=1
每个叶节点最少样本数，设置为 1 意味着允许很深的树（可能过拟合，但适合学习复杂模式）。

categorical_algorithm="RANDOM"
分类变量分裂策略为随机选择（通常用于大类变量）。

shrinkage=0.05
学习率（又名 shrinkage），控制每棵树对最终预测的贡献，越小越稳定但训练慢。

split_axis="SPARSE_OBLIQUE"
使用稀疏斜率分裂轴（Sparse Oblique Split）：允许用多个特征的线性组合进行分裂，而非一个特征一个分裂（更复杂的模型能力）。

sparse_oblique_normalization="MIN_MAX"
对线性组合中的特征进行 Min-Max 归一化。

sparse_oblique_num_projections_exponent=2.0
控制生成多少种组合分裂方式。这个值是指数，实际是 \text{num_features}^{2.0} 个组合。

num_trees=2000
训练 2000 棵树，模型越复杂，拟合能力越强。

random_seed=1234
固定随机种子，使结果可复现。

### 1. categorical_algorithm="RANDOM"
这是指定类别特征如何分裂（即决策树中如何处理字符串类型变量）的方式之一。TF-DF 支持几种算法，"RANDOM" 是其中较特殊的一种。

作用：
对于某个类别特征，树在选择分裂点时不会枚举所有可能的子集，而是随机挑选一些子集进行评估。

为什么这样做？
枚举所有子集代价非常高：如果一个特征有 20 个类别，理论上有 
2^19-1=524,287 种可能的分裂方式。

使用 "RANDOM" 算法可显著减少计算量，尤其在类别数量多的时候。

适用场景：
特征类别数多（例如名字、地名）

数据量大，训练速度重要

不追求精确最优分裂点，只求近似好结果

### 2. sparse_oblique_normalization="MIN_MAX"
这个参数是在使用 "SPARSE_OBLIQUE" 分裂轴时，指定线性组合中用到的数值特征如何归一化。

背景：SPARSE_OBLIQUE
稀疏斜率（sparse oblique）是对标准 axis-aligned 决策树的增强，它允许用多个特征的线性组合进行决策

多个特征的线性组合用于节点分裂：

$w_1 x_1 + w_2 x_2 + \cdots + w_k x_k < \text{threshold}$


其中：
- \( w_i \)：每个特征的权重
- \( x_i \)：特征值
- \( k \)：特征数量


为了让这些组合更稳定，需要对特征做标准化。

"MIN_MAX" 的含义：
每个特征会按如下方式缩放：
使用最小-最大归一化对特征 \( x \) 做缩放：

$$
x' = \frac{x - \min(x)}{\max(x) - \min(x)}
$$

归一化后有：

$$
x' \in [0, 1]
$$


这样所有特征的取值都被归一化到 [0, 1] 范围内。

目的：
让不同尺度的特征在组合时权重更加可比

避免某些取值范围大的特征主导分裂方向

可选项：
还有比如 "Z_SCORE"（标准差归一）等方式，但 MIN_MAX 更直观且适用于多数场景。

总结一句话：
categorical_algorithm="RANDOM" 是让高基数类别变量分裂更高效；

sparse_oblique_normalization="MIN_MAX" 是保证斜率分裂中多特征组合的尺度统一性。

Let's look at the model and you can also notice the information about variable importance that the model figured out

In [ ]:
model.summary()

model.summary() 是 TensorFlow / Keras 中常用的方法之一，作用是输出模型结构摘要。不过，在你使用的 TensorFlow Decision Forests (TF-DF) 中，它的作用稍有不同。

在 TF-DF 中 model.summary() 的作用：
它会打印出一个简略的模型结构信息，包括：

模型类型（如 GradientBoostedTreesModel）

树的数量、深度、节点数等

特征使用情况（例如用了哪些特征）

一些超参数设置



# Make predictions

In [ ]:
def prediction_to_kaggle_format(model, threshold=0.5):
    proba_survive = model.predict(serving_ds, verbose=0)[:,0]
    return pd.DataFrame({
        "PassengerId": serving_df["PassengerId"],
        "Survived": (proba_survive >= threshold).astype(int)
    })

def make_submission(kaggle_predictions):
    path="/kaggle/working/submission.csv"
    kaggle_predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")
    
kaggle_predictions = prediction_to_kaggle_format(model)
make_submission(kaggle_predictions)
!head /kaggle/working/submission.csv

### model.predict(serving_ds, verbose=0)
serving_ds:
一个 TensorFlow Dataset，不含标签，用于预测。一般是测试集或部署用数据。

verbose=0:
设置日志等级为“静默”，不输出任何进度条或预测日志。常用于脚本执行或 notebook 干净输出。

[:, 0] 的作用
.predict(...) 返回的是一个 NumPy 数组，形状大致为 (num_samples, num_classes)。

对于二分类问题（如 Titanic 是否生存），它返回的是每个样本的两个概率：

[[0.75, 0.25],   # 样本1：类别0的概率是0.75，类别1的概率是0.25
 [0.20, 0.80],   # 样本2：...
 ...]
 
[:, 0] 表示：只取每行的第一个值（也就是“属于类0”的概率）

# Training a model with hyperparameter tunning

Hyper-parameter tuning is enabled by specifying the tuner constructor argument of the model. The tuner object contains all the configuration of the tuner (search space, optimizer, trial and objective).


In [ ]:
tuner = tfdf.tuner.RandomSearch(num_trials=1000)
tuner.choice("min_examples", [2, 5, 7, 10])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8])

global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

#tuner.choice("use_hessian_gain", [True, False])
tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])


tuner.choice("split_axis", ["AXIS_ALIGNED"])
oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
oblique_space.choice("sparse_oblique_normalization",
                     ["NONE", "STANDARD_DEVIATION", "MIN_MAX"])
oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5])

# Tune the model. Notice the `tuner=tuner`.
tuned_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
tuned_model.fit(train_ds, verbose=0)

tuned_self_evaluation = tuned_model.make_inspector().evaluation()
print(f"Accuracy: {tuned_self_evaluation.accuracy} Loss:{tuned_self_evaluation.loss}")

## 🎯 TF-DF GBDT 模型自动调参笔记（Random Search 全注解）

使用 TensorFlow Decision Forests 的 `RandomSearch` 策略，对 Gradient Boosted Trees 模型进行超参数优化，以提升预测性能。

```python
# 初始化 Random Search Tuner，最多尝试 1000 个超参数组合
tuner = tfdf.tuner.RandomSearch(num_trials=1000)

# 基本参数定义
tuner.choice("min_examples", [2, 5, 7, 10])  # 每个叶节点的最小样本数，越小模型越复杂
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])  # 类别特征处理方式：穷举 or 随机
tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])  # 学习率
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])  # 分裂时使用的特征比例

# 树的生长策略：局部策略（LOCAL）
local_space = tuner.choice("growing_strategy", ["LOCAL"])
local_space.choice("max_depth", [3, 4, 5, 6, 8])  # 每棵树的最大深度

# 树的生长策略：全局策略（BEST_FIRST_GLOBAL）
global_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_space.choice("max_num_nodes", [16, 32, 64, 128, 256])  # 控制树的最大节点数

# 分裂方式：标准单轴分裂
tuner.choice("split_axis", ["AXIS_ALIGNED"])

# 分裂方式：斜率分裂（特征线性组合），提升表达能力
oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
oblique_space.choice("sparse_oblique_normalization", ["NONE", "STANDARD_DEVIATION", "MIN_MAX"])  # 特征归一化方式
oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])  # 权重是 0/1 还是连续值
oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5])  # 控制组合数量，指数形式

# 构建模型，并传入 tuner（开始自动调参 + 训练）
tuned_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
tuned_model.fit(train_ds, verbose=0)

# 获取模型的评估结果（accuracy 和 loss）
result = tuned_model.make_inspector().evaluation()
print(f"Accuracy: {result.accuracy}  Loss: {result.loss}")


min_examples：控制过拟合 vs 拟合能力

categorical_algorithm：类别特征分裂方式，RANDOM 更适合高基数

shrinkage：学习率，越小越稳定但训练慢

growing_strategy：决定树是用“最大深度”还是“最大节点数”控制生长

split_axis：是否使用稀疏斜率（特征组合）进行分裂，提升模型表达能力

所有 .choice(...) 都会进入搜索空间，由 RandomSearch 自动试验组合

In the last line in the cell above, you can see the accuracy is higher than previously with default parameters and parameters set by hand.

This is the main idea behing hyperparameter tuning.

For more information you can follow this tutorial: [Automated hyper-parameter tuning](https://www.tensorflow.org/decision_forests/tutorials/automatic_tuning_colab)

# Making an ensemble

Here you'll create 100 models with different seeds and combine their results

This approach removes a little bit the random aspects related to creating ML models

In the GBT creation is used the `honest` parameter. It will use different training examples to infer the structure and the leaf values. This regularization technique trades examples for bias estimates.

用于 ensemble 的模型一般更偏好“少调参 + 多样性”。

In [ ]:
predictions = None
num_predictions = 0

for i in range(100):
    print(f"i:{i}")
    # Possible models: GradientBoostedTreesModel or RandomForestModel
    model = tfdf.keras.GradientBoostedTreesModel(
        verbose=0, # Very few logs
        features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
        exclude_non_specified_features=True, # Only use the features in "features"

        #min_examples=1,
        #categorical_algorithm="RANDOM",
        ##max_depth=4,
        #shrinkage=0.05,
        ##num_candidate_attributes_ratio=0.2,
        #split_axis="SPARSE_OBLIQUE",
        #sparse_oblique_normalization="MIN_MAX",
        #sparse_oblique_num_projections_exponent=2.0,
        #num_trees=2000,
        ##validation_ratio=0.0,
        random_seed=i,
        honest=True,
    )
    model.fit(train_ds)
    
    sub_predictions = model.predict(serving_ds, verbose=0)[:,0]
    if predictions is None:
        predictions = sub_predictions
    else:
        predictions += sub_predictions
    num_predictions += 1

predictions/=num_predictions

kaggle_predictions = pd.DataFrame({
        "PassengerId": serving_df["PassengerId"],
        "Survived": (predictions >= 0.5).astype(int)
    })

make_submission(kaggle_predictions)

### 什么是诚实分裂（Honest Trees）？
 标准分裂（非诚实）：
所有样本既用来决定如何分裂节点（建树），也用来计算每个叶节点的预测值。

 诚实分裂（honest=True）：
样本会被随机拆分为两部分：

结构集（structure set）：用于决定如何分裂

估计集（estimation set）：用于计算叶节点输出值（如类别概率）

### 为什么要这么做？
优点：
避免信息泄露：建树用的数据不再参与预测估计，减少过拟合。

更稳健的预测值：因为预测值不是对训练集的“回声拟合”，而是独立估计。

在集成（多个树）时特别有效：每棵树都更“谦逊”，合起来泛化能力强。

# What is next

If you want to learn more about TensorFlow Decision Forests and its advanced features, you can follow the official documentation [here](https://www.tensorflow.org/decision_forests) 